In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *

In [2]:
from data_split import *

In [3]:
ml_data = pd.read_pickle("/home/seho/Passenger_Demand/data/ml_data.pkl")

In [4]:
ml_data = ml_data.set_index(["mybi_stop_id", "stop_nm", "transdate"])

In [13]:
ml_data["hour"] = ml_data["hour"].astype(str)

In [14]:
splitted_data = split(X_data = ml_data.drop(["normalcnt_bf1d", "studentcnt_bf1d", "childcnt_bf1d", 
                                             "normalcnt_bf1w", "studentcnt_bf1w", "childcnt_bf1w", 
                                             "longitude", "latitude", "day", "totalcnt", "studentcnt", "childcnt", "normalcnt"], 1),
                      Y_data = ml_data["totalcnt"],
                      split_ratio = [0.5, 0.3, 0.2], 
                      seed = 1234)

In [15]:
train_data = pd.concat([splitted_data["X_train"], splitted_data["Y_train"]], 1)

In [16]:
train_data.shape

(372563, 115)

In [17]:
reg_experiment = setup(train_data, 
                       target = "totalcnt", 
#                        log_experiment = True, 
                       experiment_name="Passenger_Demand")

,Description,Value
0,session_id,3746
1,Target,totalcnt
2,Original Data,"(372563, 115)"
3,Missing Values,True
4,Numeric Features,14
5,Categorical Features,100
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(260794, 501)"


In [18]:
bese_model = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.4002,7.2157,2.6862,0.7961,0.4663,0.5689,2.7200
rf,Random Forest Regressor,1.3986,7.6230,2.7610,0.7846,0.4646,0.5881,135.1600
gbr,Gradient Boosting Regressor,1.4745,7.8843,2.8078,0.7772,0.4875,0.5940,35.0780
et,Extra Trees Regressor,1.4354,7.9922,2.8270,0.7742,0.4774,0.6162,223.0600
lr,Linear Regression,1.5858,8.2695,2.8756,0.7663,0.5335,0.6081,2.3560
ridge,Ridge Regression,1.5858,8.2695,2.8756,0.7663,0.5335,0.6081,0.6500
br,Bayesian Ridge,1.5833,8.2696,2.8757,0.7663,0.5324,0.6071,10.5100
omp,Orthogonal Matching Pursuit,1.5842,8.2803,2.8775,0.7660,0.5329,0.6062,0.6580
en,Elastic Net,1.5570,8.5096,2.9171,0.7595,0.5181,0.5898,0.6500
lasso,Lasso Regression,1.5741,8.5387,2.9221,0.7587,0.5237,0.5849,0.7500


In [2]:
%%time
mybicard = pd.read_parquet('/home/seho/Passenger_Demand/data/mybicard.parquet', engine='pyarrow')

CPU times: user 50.3 s, sys: 32.3 s, total: 1min 22s
Wall time: 34.5 s


In [ ]:
ml_data = pd.read_parquet("/home/seho/Passenger_Demand/data")

In [ ]:
reg_experiment = setup(housing_data, 
                       target = 'median_house_value', 
                       session_id=123, 
                       log_experiment=True, 
                       experiment_name='ca_housing')